In [7]:
from twython import Twython, TwythonStreamer
from auth import *
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pyodbc
import sys
from pprint import pprint

analyser = SentimentIntensityAnalyzer()

conn = pyodbc.connect(r'DSN=adventureDSN;UID=python;PWD=password')
cur = conn.cursor()

class MyTwythonStreamer(TwythonStreamer):
    def on_success(self, data):
        tweets = []
        if data['lang'] == 'en':  # Only taking english tweets
            tweets.append(data)
            print('Tweet {}'.format(len(tweets)))
            for i in range(0,len(tweets)):
                worked = 0
                try:
                    full_text = str(tweets[i]['extended_tweet']['full_text'])
                    sentiment = analyser.polarity_scores(full_text)['compound']
                    worked = 1
                except:
                    pass
                try:
                    full_text = str(tweets[i]['quoted_status']['extended_tweet']['full_text'])
                    sentiment = analyser.polarity_scores(full_text)['compound']
                    worked = 1
                except:
                    pass
                try:
                    full_text = str(tweets[i]['retweeted_status']['extended_tweet']['full_text'])
                    sentiment = analyser.polarity_scores(full_text)['compound']
                    worked = 1
                except:
                    pass
                finally:
                    if worked == 0:
                        full_text = str(tweets[i]['text'])
                        sentiment = analyser.polarity_scores(full_text)['compound']
                
                followers = int(tweets[i]['user']['followers_count'])
                favourites = int(tweets[i]['user']['favourites_count'])
                verified = str(tweets[i]['user']['verified'])
                status_count = int(tweets[i]['user']['statuses_count'])
                timestamp = int(tweets[i]['timestamp_ms'])
                insert_sql = 'INSERT INTO [dbo].[politics] (sentiment, followers_count, favourites_count, verified, statuses_count, timestamp_ms, [text]) VALUES (?, ?, ?, ?, ?, ?, ?)'
                cur.execute(insert_sql, sentiment, followers, favourites, verified, status_count, timestamp, full_text)
                cur.commit()
        if len(tweets) >= 10:
            self.disconnect()
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()

stream = MyTwythonStreamer(consumer_key, consumer_secret, access_token, access_token_secret)
stream.statuses.filter(track='labour')

Tweet 1
woo
Tweet 1
woo
Tweet 1
woo
Tweet 1
woo
Tweet 1
woo
Tweet 1
woo
Tweet 1
woo
Tweet 1
woo
Tweet 1
woo
Tweet 1
woo
Tweet 1
woo
Tweet 1
woo
Tweet 1
woo
Tweet 1
woo
Tweet 1
woo
Tweet 1
woo


KeyboardInterrupt: 